In [1]:
import re
def preprocess_code_line(code, remove_python_common_tokens=False):
    code = code.replace('(', ' ').replace(')', ' ').replace('{', ' ').replace('}', ' ').replace('[', ' ').replace(']',
                                                                                                                  ' ').replace(
        '.', ' ').replace(':', ' ').replace(';', ' ').replace(',', ' ').replace(' _ ', '_')

    code = re.sub('``.*``', '<STR>', code)
    code = re.sub("'.*'", '<STR>', code)
    code = re.sub('".*"', '<STR>', code)
    code = re.sub('\d+', '<NUM>', code)

    code = code.split()
    code = ' '.join(code)
    if remove_python_common_tokens:
        new_code = ''
        python_common_tokens = []
        for tok in code.split():
            if tok not in [python_common_tokens]:
                new_code = new_code + tok + ' '

        return new_code.strip()

    else:
        return code.strip()

In [2]:
def hunk_empty(hunk):
    if hunk.strip() == '':
        return True

def get_hunk_from_diff(diff):
    hunk_list = []
    hunk = ''
    for line in diff.splitlines():
        if line.startswith(('+', '-')): 
            hunk += line.strip() + '\n'
        else:
            if not hunk_empty(hunk): # finish a hunk 
                hunk = hunk [:-1]
                hunk_list.append(hunk)
                hunk = ''

    if not hunk_empty(hunk): 
        hunk_list.append(hunk)
    
    return hunk_list


In [4]:
import pandas as pd 
df = pd.read_csv('codejit_time_suffer.csv',low_memory=False)

In [11]:
cms = set(df['commit_id'])
len(cms)

20274

In [25]:
hunk_map = dict()
for cm in cms:
    tmp_df = df[df['commit_id']==cm]
    hunks_cm = list()
    for _, row in tmp_df.iterrows():
        diff = row['diff']
        hunks = get_hunk_from_diff(diff)
        for hunk in hunks:
            hunk_tokens = list()
            for line in hunk.splitlines():
                if line.startswith('+'):
                    hunk_tokens += ["ADD"]
                if line.startswith('-'):
                    hunk_tokens += ["DEL"]
                line = preprocess_code_line(line[1:])
                hunk_tokens += line.split()
            hunks_cm.append(' '.join(hunk_tokens))
    hunk_map[cm] = hunks_cm

In [26]:
hunk_map['e00499eb4c80752b2c464f3a98bf0b6ce7b1e212']
import pickle 
pickle.dump(hunk_map,open('hunk_map.pkl','wb'))

['DEL if ret = ff_alloc_packet<NUM> avctx pkt mb_width*mb_height*MAX_MB_BYTES + FF_MIN_BUFFER_SIZE < <NUM> ADD if ret = ff_alloc_packet<NUM> avctx pkt mb_width* int<NUM>_t mb_height*MAX_MB_BYTES + FF_MIN_BUFFER_SIZE < <NUM>']

In [27]:
import pickle 
pickle.dump(hunk_map,open('hunk_map.pkl','wb'))

In [16]:
ll = [len(v) for k,v in hunk_map.items()]

In [17]:
max(ll)

5699

In [18]:
min(ll)

1

In [20]:
sum(ll)/len(ll)

14.58833974548683

In [30]:
hunk_map_sep = dict()
for cm in cms:
    tmp_df = df[df['commit_id']==cm]
    hunks_cm = list()
    for _, row in tmp_df.iterrows():
        diff = row['diff']
        hunks = get_hunk_from_diff(diff)
        for hunk in hunks:
            hunk_tokens = list()
            hunk_tokens += ["ADD"]
            for line in hunk.splitlines():
                if line.startswith('-'):
                    continue
                line = preprocess_code_line(line[1:])
                hunk_tokens += line.split()
            hunk_tokens += ["DEL"]
            for line in hunk.splitlines():
                if line.startswith('+'):
                    continue
                line = preprocess_code_line(line[1:])
                hunk_tokens += line.split()
            hunks_cm.append(' '.join(hunk_tokens))
    hunk_map_sep[cm] = hunks_cm

In [ ]:
import pickle 
pickle.dump(hunk_map_sep,open('hunk_map_sep.pkl','wb'))

In [31]:
hunk_map_sep['e00499eb4c80752b2c464f3a98bf0b6ce7b1e212']


['ADD if ret = ff_alloc_packet<NUM> avctx pkt mb_width* int<NUM>_t mb_height*MAX_MB_BYTES + FF_MIN_BUFFER_SIZE < <NUM> DEL if ret = ff_alloc_packet<NUM> avctx pkt mb_width*mb_height*MAX_MB_BYTES + FF_MIN_BUFFER_SIZE < <NUM>']